In [ ]:
import time

from IPython.display import display
import arrow
import base_node
import base_node_rpc as bnr
import base_node_rpc.bootloader_driver
import dropbot as db
import hv_switching_board as hv
import hv_switching_board.driver
import logging; logging.basicConfig(level=logging.DEBUG)
import numpy as np
import platformio_helpers as pioh
import serial_device

# Show available COM ports.
display(serial_device.comports(only_available=True))

try:
    proxy.terminate()
except:
    pass
proxy = db.SerialProxy()
display(proxy.properties)
i2c_devices = proxy.i2c_scan()
print 'Found I2C devices at the following addresses: %s' % i2c_devices

# Find HV switching board firmware.
firmware_path = pioh.conda_bin_path().joinpath('hv-switching-board', 'v3_1',
                                               'firmware.hex')
print 'Found firmware: %s (%s)' % (firmware_path, arrow.get(firmware_path.mtime).to('local'))


def set_uuid(bootloader, value):
    '''
    Parameters
    ----------
    value : uuid.UUID
    '''
    config_bytes = bootloader.read_eeprom(0, base_node.driver.CONFIG_DTYPE.itemsize)
    config = np.fromstring(config_bytes.tostring(), dtype=base_node.driver.CONFIG_DTYPE)[0]
    config['uuid'] = value.bytes
    bootloader.write_eeprom(0, config.tobytes())
    
    
def set_board_uuid(hv_board, value):
    bootloader = base_node_rpc.bootloader_driver.TwiBootloader(hv_board.proxy)
    hv_board.reboot_recovery()
    set_uuid(bootloader, value)
    bootloader.start_application()

    
def write_firmware(proxy):
    i2c_devices = proxy.i2c_scan()
    switching_board_addrs = [address for address in i2c_devices if address not in [10, 81]]

    # Create object to interact with I2C bootloader.
    bootloader = bnr.bootloader_driver.TwiBootloader(proxy, bootloader_address=0x29)
    if not switching_board_addrs:
        raise IOError('No switching boards found on bus.')
    elif bootloader.bootloader_address in switching_board_addrs:
        print 'I2C bootloader found on bus at address 41.'
    else:
        selection = None
        while selection not in switching_board_addrs:
            print ('Please select one of the following switching boards into '
                   'recovery mode (or enter "q" to quit): %s' %
                   switching_board_addrs)

            selection = raw_input()
            if selection == 'q':
                return
            try:
                selection = int(selection)
            except:
                pass
        if selection in switching_board_addrs:
            hv_board = hv.driver.HVSwitchingBoard(proxy, selection)
            hv_board.reboot_recovery()
            time.sleep(.1)

    bootloader.write_firmware(firmware_path)
    logging.info('Launch firmware')
    bootloader.start_application()
    # Wait for switching board firmware to boot.
    time.sleep(.1)
    i2c_devices = proxy.i2c_scan()
    print 'Found I2C devices at the following addresses: %s' % i2c_devices
    return selection

-----------------------------------------------


# Flash firmware

In [ ]:
selection = write_firmware(proxy)

-----------------------------------------------


# Set switching board UUID

In [ ]:
import uuid

assert(selection is not None)
hv_board = hv_switching_board.HVSwitchingBoard(proxy, address=selection)
hv_uuid = uuid.UUID('XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX')
set_board_uuid(hv_board, hv_uuid)
time.sleep(.1)
print 'Set UUID to:', hv_board.uuid

-----------------------------------------------


# Set switching board I2C address

In [ ]:
assert(selection is not None)
hv_board = hv_switching_board.HVSwitchingBoard(proxy, address=selection)
hv_board.set_i2c_address(32)